# Mikä on paras päivä sijoittaa osakemarkkinoille (Vanguard 500 Index Fund ETF)

-------------------------------------------------------------------------------------------------------------------------------------------------
## Ongelma

### Sijoittajana etsin aina tapoja optimoida tuottoni ETF-sijoituksilleni. Yksi tärkeimmistä sijoittamisen näkökohdista on ymmärtää osakemarkkinoiden käyttäytyminen. Halusin tietää, onko VOO ETF:n historiallisissa tiedoissa kuvioita, joita voitaisiin hyödyntää sijoitusstrategioideni parantamiseksi. Vastatakseni tähän kysymykseen kirjoitin Python-ohjelman, joka suorittaa yksityiskohtaisen data-analyysin ja jälkitestauksen simulaation historiallisille VOO ETF -tiedoille. Ohjelma käyttää kahta menetelmää tietojen analysointiin:

-------------------------------------------------------------------------------------------------------------------------------------------------
## Ratkaisun etsintä kahdella menetelmällä

## Menetelmä 1: Avaushinnan muutoksen laskeminen

#### Tämä menetelmä sisältää ETF:n avaushinnan prosentuaalisen muutoksen laskemisen edellisestä päivästä. Tämän avulla voimme saada käsityksen siitä, kuinka epävakaa ETF on päivittäin. Sen jälkeen ohjelma ryhmittelee tiedot viikonpäivien mukaan ja etsii prosentuaalisen muutoksen keskiarvon kullekin päivälle. Tämä antaa meille käsityksen siitä, mitkä viikonpäivät vaihtelevat eniten, ja näin ollen niihin voi olla riskialtisempaa sijoittaa.

## Menetelmä 2: Täysi kalenteri ja jälkitestaus

#### Tässä menetelmässä luodaan kokonainen kalenteri päivämääristä tietojen alkamis- ja lopetuspäivän väliin. Ohjelma yhdistää tämän kalenterin ETF-tietoihin ja täyttää kaikki puuttuvat tiedot täyttämällä edellisen saatavilla olevan päivämäärän perusteella. Tämän avulla voimme simuloida sijoittamista minä tahansa viikonpäivänä, myös viikonloppuisin, kun pörssi on kiinni. Kun tiedot on kohdistettu, se eliminoi viikonloput ja simuloi ETF:n ostamista tiettynä päivänä ja sen myymistä saatavilla olevien tietojen viimeisenä päivänä sekä laskee myös lopullisen investointimäärän kullekin viikonpäivälle.

In [48]:
import pandas as pd
import os
import logging
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

logging.getLogger('prophet').setLevel(logging.WARNING)

In [49]:
data_import = pd.read_csv("VOO.csv", parse_dates=["Date"])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720131,8600
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747894,33750
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731789,59400
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250
...,...,...,...,...,...,...,...
2876,2022-02-10,414.929993,420.799988,411.140015,413.179993,413.179993,9260500
2877,2022-02-11,413.149994,415.200012,403.570007,404.940002,404.940002,10452700
2878,2022-02-14,404.429993,405.989990,400.239990,403.619995,403.619995,10162000
2879,2022-02-15,408.179993,410.290009,407.459991,410.100006,410.100006,6822300


In [50]:
day_mapper = {0: "Maanantai", 1:"Tiistai", 2:"Keskiviikko", 3:"Torstai", 4:"Perjantai", 5:"Lauantai", 6:"Sunnuntai"}
data_import["Viikonpäivä"] = data_import["Date"].map(lambda x: day_mapper[x.dayofweek])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,Viikonpäivä
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500,Torstai
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720131,8600,Perjantai
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747894,33750,Maanantai
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731789,59400,Tiistai
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250,Keskiviikko
...,...,...,...,...,...,...,...,...
2876,2022-02-10,414.929993,420.799988,411.140015,413.179993,413.179993,9260500,Torstai
2877,2022-02-11,413.149994,415.200012,403.570007,404.940002,404.940002,10452700,Perjantai
2878,2022-02-14,404.429993,405.989990,400.239990,403.619995,403.619995,10162000,Maanantai
2879,2022-02-15,408.179993,410.290009,407.459991,410.100006,410.100006,6822300,Tiistai


# Menetelmä 1

In [51]:
data_import["erotus_edellisestä_pv"] = (data_import["Open"].diff() / data_import["Open"]) * 100
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,Viikonpäivä,erotus_edellisestä_pv
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500,Torstai,NaN
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720131,8600,Perjantai,-0.806452
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747894,33750,Maanantai,1.243200
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731789,59400,Tiistai,-0.116689
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250,Keskiviikko,-0.214376
...,...,...,...,...,...,...,...,...,...
2876,2022-02-10,414.929993,420.799988,411.140015,413.179993,413.179993,9260500,Torstai,-0.872435
2877,2022-02-11,413.149994,415.200012,403.570007,404.940002,404.940002,10452700,Perjantai,-0.430836
2878,2022-02-14,404.429993,405.989990,400.239990,403.619995,403.619995,10162000,Maanantai,-2.156121
2879,2022-02-15,408.179993,410.290009,407.459991,410.100006,410.100006,6822300,Tiistai,0.918712


In [52]:
data_import.groupby("Viikonpäivä")["erotus_edellisestä_pv"].mean()

Viikonpäivä
Keskiviikko    0.076498
Maanantai     -0.034693
Perjantai      0.064210
Tiistai        0.103887
Torstai       -0.001912
Name: erotus_edellisestä_pv, dtype: float64

## Tämä menetelmä auttoi minua ymmärtämään ETF:n päivittäisen volatiliteetin ja sen, millä viikonpäivillä volatiliteetti on yleensä enemmän, mihin voisi olla riskialtisempaa sijoittaa. Kuten huomaamme volatiliteetti on pienin torstaisin ja suurin tiistaisin.

_________________________________________________________________________________________________________________________________________________

# Menetelmä 2

In [53]:
dates = pd.date_range(start=data_import["Date"].min(), end=data_import["Date"].max())

date_table = pd.DataFrame(data={"Kalenteri Pv":dates})
date_table["Viikonpäivä"] = date_table["Kalenteri Pv"].map(lambda x: day_mapper[x.dayofweek])
date_table

,Kalenteri Pv,Viikonpäivä
0,2010-09-09,Torstai
1,2010-09-10,Perjantai
2,2010-09-11,Lauantai
3,2010-09-12,Sunnuntai
4,2010-09-13,Maanantai
...,...,...
4174,2022-02-12,Lauantai
4175,2022-02-13,Sunnuntai
4176,2022-02-14,Maanantai
4177,2022-02-15,Tiistai


In [54]:
full_calendar = pd.merge(left= date_table, right= data_import, how='left', left_on = 'Kalenteri Pv', right_on = "Date")
full_calendar

,Kalenteri Pv,Viikonpäivä_x,Date,Open,High,Low,Close,Adj Close,Volume,Viikonpäivä_y,erotus_edellisestä_pv
0,2010-09-09,Torstai,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500.0,Torstai,NaN
1,2010-09-10,Perjantai,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720131,8600.0,Perjantai,-0.806452
2,2010-09-11,Lauantai,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-09-12,Sunnuntai,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-09-13,Maanantai,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747894,33750.0,Maanantai,1.243200
...,...,...,...,...,...,...,...,...,...,...,...
4174,2022-02-12,Lauantai,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4175,2022-02-13,Sunnuntai,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4176,2022-02-14,Maanantai,2022-02-14,404.429993,405.989990,400.239990,403.619995,403.619995,10162000.0,Maanantai,-2.156121
4177,2022-02-15,Tiistai,2022-02-15,408.179993,410.290009,407.459991,410.100006,410.100006,6822300.0,Tiistai,0.918712


In [55]:
full_calendar = pd.merge(left= date_table, right= data_import, how='left', left_on = 'Kalenteri Pv', right_on = "Date")
full_calendar = full_calendar[~full_calendar["Viikonpäivä_x"].isin(["Lauantai", "Sunnuntai"])]
full_calendar.rename(columns={"Date":"Pörssi Pv", "Open":"Avaus", "High":"Ylin", "Low":"Alin", "Close":"Päätös", "Adj Close":"Oik. Päätös", "Volume":"Vaihto"}, inplace=True)
full_calendar

,Kalenteri Pv,Viikonpäivä_x,Pörssi Pv,Avaus,Ylin,Alin,Päätös,Oik. Päätös,Vaihto,Viikonpäivä_y,erotus_edellisestä_pv
0,2010-09-09,Torstai,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500.0,Torstai,NaN
1,2010-09-10,Perjantai,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720131,8600.0,Perjantai,-0.806452
4,2010-09-13,Maanantai,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747894,33750.0,Maanantai,1.243200
5,2010-09-14,Tiistai,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731789,59400.0,Tiistai,-0.116689
6,2010-09-15,Keskiviikko,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250.0,Keskiviikko,-0.214376
...,...,...,...,...,...,...,...,...,...,...,...
4172,2022-02-10,Torstai,2022-02-10,414.929993,420.799988,411.140015,413.179993,413.179993,9260500.0,Torstai,-0.872435
4173,2022-02-11,Perjantai,2022-02-11,413.149994,415.200012,403.570007,404.940002,404.940002,10452700.0,Perjantai,-0.430836
4176,2022-02-14,Maanantai,2022-02-14,404.429993,405.989990,400.239990,403.619995,403.619995,10162000.0,Maanantai,-2.156121
4177,2022-02-15,Tiistai,2022-02-15,408.179993,410.290009,407.459991,410.100006,410.100006,6822300.0,Tiistai,0.918712


In [56]:
full_calendar = full_calendar.bfill(axis='rows').reset_index(drop=True)
full_calendar.to_clipboard()

In [57]:
full_calendar["Viikonpäivä_x"].value_counts()

Torstai        597
Perjantai      597
Maanantai      597
Tiistai        597
Keskiviikko    597
Name: Viikonpäivä_x, dtype: int64

In [58]:
def day_backtester(day, amount_to_invest, data):
    temp_data = data[data["Viikonpäivä_x"] == day]
    temp_data["Shares Owned"] = amount_to_invest / temp_data["Avaus"]
    final_price = temp_data['Avaus'].iloc[-1]
    final_amout = temp_data["Shares Owned"].sum() * final_price
    formatted_final_amout = "${:,.2f}".format(final_amout)
    return formatted_final_amout

In [60]:
pd.options.mode.chained_assignment = None  # default='warn'
for i in full_calendar["Viikonpäivä_x"].unique():
    print(i, day_backtester(i, 200, full_calendar))

Torstai $263,240.28
Perjantai $261,951.70
Maanantai $256,427.44
Tiistai $258,642.15
Keskiviikko $258,407.44


### Tällä menetelmällä pystyin simuloimaan sijoittamista minä tahansa viikonpäivänä, myös viikonloppuisin, kun osakemarkkinat ovat kiinni. Ohjelma laski myös loppusijoituksen määrän jokaiselle viikonpäivälle, mikä auttoi minua tietämään eri päivien sijoitetun pääoman tuoton. Jos olisin sijoittanut pääomani torstaisin olisi pääomani suurin mahdollinen.

##### Täytyy ottaa huomioon, että ohjelma on ainoastaan menneen datan tutkimusta ja analysointia, eikä ohjelman tuloksia ole tarkoitettu sijoitusneuvoksi tai ohjeeksi. Ohjelman käyttötarkoitus on stimuloida ohjelman kirjoittaneen henkilön taitoja käsitellä dataa sekä sen sisältöä

### Yhteenvetona voidaan todeta, että tämä ohjelma on suunniteltu auttamaan sijoittajia optimoimaan sijoituksensa tuottoa VOO ETF:ssä analysoimalla historiallisia tietoja ja simuloimalla erilaisia sijoitusstrategioita. Käyttämällä kahta menetelmää, laskemalla avaushinnan muutosta sekä jälkitestausta, ohjelma tarjoaa arvokasta tietoa osakemarkkinoiden käyttäytymisestä valitulle sijoituskohteelle. Ohjelman kyky simuloida sijoituksia minä tahansa viikonpäivänä, mukaan lukien viikonloppuisin ja pyhäpäivinä, tekee siitä tehokkaan työkalun kysyisen volatiliteetin ymmärtämiseen ja tietoon perustuvien sijoituspäätösten tekemiseen. 

### Ohjelma ratkaisi ongelmat, jotka liittyivät sijoituskohteen historiallisen käyttäytymisen ymmärtämisen puutteeseen, sijoitusstrategioiden simulointiin sekä ETF:n volatiliteetin ymmärtämiseen liittyviin ongelmiin. Kaiken kaikkiaan tämä ohjelma on arvokas työkalu kaikille, jotka ovat kiinnostuneita maksimoimaan Vanguard 500 Index Fund ETF sijoitetun pääoman tuoton.

Kiitos ohjelman käytöstä.